In [3]:
import seaborn as sns
from numpy import dstack
from pandas import read_csv
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pydot
import graphviz

In [4]:
def file_load(filepath):
    df = read_csv(filepath, header=None, delim_whitespace=True)
    return df.values    

def train_test_append(filenames, append_before=''):
    datalist = list()
    for name in filenames:
        data = file_load(append_before + name)
        datalist.append(data)
    datalist = dstack(datalist)
    return datalist

def inertial_signals_load(group, append_before=''):
    filepath = append_before + group + '/Inertial Signals/'
    filenames = list()
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    X = train_test_append(filenames, filepath)
    y = file_load(append_before + group + '/y_'+group+'.txt')
    return X, y
    

def load_dataset(append_before=''):
    trainX, trainy = inertial_signals_load('train')
    testX, testy = inertial_signals_load('test')
    trainy = trainy - 1
    testy = testy - 1
    # trainy = to_categorical(trainy)
    # testy = to_categorical(testy)
    train=(trainX, trainy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [5]:
trainX, trainy, testX, testy = load_dataset()
verbose, epochs, batch_size =1, 30, 192 #192
n_timesteps = trainX.shape[1]
n_features = trainX.shape[2]
n_outputs = trainy.shape[1]
n_steps = 4
n_length = 32
trainX = trainX.reshape((trainX.shape[0], 1152))
testX = testX.reshape((testX.shape[0], 1152)) 
print(trainX.shape,testX.shape)

(7352, 128, 9) (7352, 1) (2947, 128, 9) (2947, 1)
(7352, 1152) (2947, 1152)


In [6]:
#SVM model
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import metrics
model = SVC()
model.fit(trainX,trainy)
history=model.fit(trainX, trainy)
predictions = model.predict(testX)
print('SVM',accuracy_score(testy,predictions)) #0.887343
clr=metrics.classification_report(testy, predictions)
print(clr)

c:\Users\19581\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\19581\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVM 0.8873430607397353
              precision    recall  f1-score   support

           0       0.87      0.96      0.91       496
           1       0.89      0.87      0.88       471
           2       0.87      0.86      0.86       420
           3       0.85      0.77      0.81       491
           4       0.84      0.86      0.85       532
           5       1.00      1.00      1.00       537

    accuracy                           0.89      2947
   macro avg       0.89      0.89      0.88      2947
weighted avg       0.89      0.89      0.89      2947



In [7]:
#DT model
from sklearn import tree #导入需要的模块
clf = tree.DecisionTreeClassifier() #实例化模型对象
from sklearn import metrics

clf = clf.fit(trainX,trainy) #用训练集数据训练模型
result = clf.score(testX,testy) #对我们训练的模型精度进行打分
print('DT',result) 
predictions = clf.predict(testX)
clr=metrics.classification_report(testy, predictions)
print(clr)

DT 0.7298948082796064
              precision    recall  f1-score   support

           0       0.60      0.61      0.61       496
           1       0.67      0.59      0.63       471
           2       0.59      0.60      0.60       420
           3       0.74      0.73      0.73       491
           4       0.73      0.79      0.76       532
           5       1.00      1.00      1.00       537

    accuracy                           0.73      2947
   macro avg       0.72      0.72      0.72      2947
weighted avg       0.73      0.73      0.73      2947



In [8]:
#KNN model
from sklearn import datasets       #导入数据模块
from sklearn.model_selection import train_test_split     #导入切分训练集、测试集模块
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics

knn = KNeighborsClassifier(n_neighbors=2)    #实例化KNN模型 n_neighbors=2效果最佳
knn.fit(trainX, trainy)      #放入训练数据进行训练
predictions=knn.predict(testX)           #打印预测内容
print('KNN',accuracy_score(testy,predictions))
clr=metrics.classification_report(testy, predictions)
print(clr)

#查看n_neighbors=多少效果最好
# from sklearn.model_selection  import cross_val_score
# k_range = range(1, 100)
# k_error = []
# for k in k_range:
#     knn = KNeighborsClassifier(n_neighbors=k)
#     #cv参数决定数据集划分比例，这里是按照5:1划分训练集和测试集
#     scores = cross_val_score(knn, trainX, trainy, cv=6, scoring='accuracy')
#     k_error.append(1 - scores.mean())
# #画图，x轴为k值，y值为误差值
# plt.plot(k_range, k_error)
# plt.xlabel('Value of K for KNN')
# plt.ylabel('Error')
# plt.show()

c:\Users\19581\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neighbors\_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNN 0.7485578554462164
              precision    recall  f1-score   support

           0       0.93      0.73      0.82       496
           1       0.96      0.75      0.84       471
           2       0.99      0.36      0.53       420
           3       0.61      0.82      0.70       491
           4       0.48      0.75      0.58       532
           5       1.00      1.00      1.00       537

    accuracy                           0.75      2947
   macro avg       0.83      0.74      0.75      2947
weighted avg       0.82      0.75      0.75      2947



'''
SVM: 88.73%
DT: 72.61%
KNN: 74.85%
CNN only: 91.75%
LSTM only: 91.89%
CNN-LSTM: 92.89%
LSTM-CNN: 95.28%
'''